In [8]:
from pymavlink import mavutil
import time
from datetime import datetime, timezone
from influxdb import InfluxDBClient


In [9]:
master = mavutil.mavlink_connection('udp:localhost:14550')
master.wait_heartbeat()
time.sleep(2)

In [10]:
client = InfluxDBClient(host='192.168.0.99', port=8086)

In [11]:
client.create_database('pyexample')
print(client.get_list_database())
client.switch_database('pyexample')

[{'name': '_internal'}, {'name': 'pyexample'}]


In [14]:
while True:
    master.wait_heartbeat()

    # Get data from mavlink
    mavlink_data = master.recv_match(type=['VFR_HUD'])

    # local time in RFC 3339 format
    local_time = datetime.now(timezone.utc).astimezone().isoformat()
    #print(local_time)

    dict_body = {}
    dict_body["measurement"] = mavlink_data._type
    dict_body["time"] = local_time
    dict_body["fields"] = mavlink_data.to_dict()

    json_body = [dict_body]

    #print(json_body)
    client.write_points(json_body)

    time.sleep(1)

KeyboardInterrupt: 

In [7]:
master.close()

In [15]:
master.wait_heartbeat()

In [17]:
print(mavlink_data)

ATTITUDE {time_boot_ms : 1438720, roll : -0.0018581367330625653, pitch : -0.05388554930686951, yaw : -0.20174238085746765, rollspeed : -0.02288021333515644, pitchspeed : 0.0014760567573830485, yawspeed : -0.0032845246605575085}
